# CS Programs ChromaDB Query Tool - 使用默认嵌入模型

**不需要API密钥，使用ChromaDB自带的默认嵌入函数**

- 作者: Assistant
- 日期: 2025-09-16
- 特点: 无需API调用，使用ChromaDB内置嵌入模型

In [1]:
# Cell 1: 导入必要的库
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import json
import os
from pprint import pprint
from typing import List, Dict, Any

print("✅ 库导入成功!")
print(f"ChromaDB版本: {chromadb.__version__}")

✅ 库导入成功!
ChromaDB版本: 1.0.21


In [2]:
# Cell 2: 初始化数据库和默认嵌入函数
DB_PATH = "vectors"
COLLECTION_NAME = "cs_programs_default"

# 创建持久化客户端
chroma_client = chromadb.PersistentClient(path=DB_PATH)

# 使用ChromaDB默认嵌入函数（不需要API）
default_ef = embedding_functions.DefaultEmbeddingFunction()

print(f"✅ 成功初始化ChromaDB客户端，数据库路径: {DB_PATH}")
print(f"✅ 使用默认嵌入函数: {type(default_ef).__name__}")
print(f"📁 集合名称: {COLLECTION_NAME}")

✅ 成功初始化ChromaDB客户端，数据库路径: vectors
✅ 使用默认嵌入函数: DefaultEmbeddingFunction
📁 集合名称: cs_programs_default


In [3]:
# Cell 3: 数据处理函数
def process_admission_data(admission_data_str: str) -> str:
    """处理录取数据字符串"""
    if not admission_data_str or admission_data_str == 'null':
        return "暂无录取案例数据"
    
    try:
        # 处理字符串格式的列表转换
        data_str = admission_data_str.replace("'", '"')
        data_list = json.loads(data_str)
        
        cases = []
        for case in data_list:
            case_text = f"录取案例({case.get('录取时间', 'N/A')}): {case.get('录取结果', 'N/A')}。"
            case_text += f"申请者背景: {case.get('学校（档次）', 'N/A')} {case.get('本科专业', 'N/A')}专业，"
            case_text += f"GPA/Rank {case.get('GPA/Rank', 'N/A')}，"
            case_text += f"科研经历 {case.get('科研经历', 'N/A')}，"
            case_text += f"实习经历 {case.get('实习经历', 'N/A')}。"
            if case.get('其他（语言/推荐信）'):
                case_text += f"其他信息: {case.get('其他（语言/推荐信）', '')}"
            cases.append(case_text)
        
        return " ".join(cases)
    except (json.JSONDecodeError, TypeError, AttributeError):
        return f"录取数据: {admission_data_str}"

def create_document_text(row: pd.Series) -> str:
    """创建用于语义搜索的富文本文档"""
    admission_text = process_admission_data(row['admission_data'])
    
    document = f"""
项目名称: {row['program_name']}
所属大学: {row['university']}
地区: {row['region']}
项目等级: {row['tier']}
学制: {row['duration']}
授课语言: {row['language']}
学位类型: {row['degree_type']}

项目优点: {row['pros']}

项目缺点: {row['cons']}

招生偏好: {row['admission_preference']}

申请注意事项: {row['application_notes']}

奖学金信息: {row['scholarship']}

过往录取案例: {admission_text}

其他信息: {row['other_info']} {row['other_notes']}
    """
    return document.strip()

print("✅ 数据处理函数定义完成!")

✅ 数据处理函数定义完成!


In [4]:
# Cell 4: 加载和处理CSV数据
CSV_PATH = "global_cs_programs.csv"

if not os.path.exists(CSV_PATH):
    print(f"❌ CSV文件不存在: {CSV_PATH}")
    print("请确保文件在当前目录下")
else:
    # 读取CSV数据
    df = pd.read_csv(CSV_PATH).fillna('')
    print(f"📊 读取到 {len(df)} 个CS项目")
    
    # 显示数据概览
    print(f"\n📋 数据概览:")
    print(f"DataFrame shape: {df.shape}")
    print(f"\n列名:")
    pprint(list(df.columns))
    print(f"\n前2行预览:")
    print(df[['program_name', 'university', 'region', 'tier']].head(2))

📊 读取到 76 个CS项目

📋 数据概览:
DataFrame shape: (76, 18)

列名:
['program_name',
 'university',
 'region',
 'tier',
 'duration',
 'language',
 'degree_type',
 'internship_required',
 'thesis_required',
 'scholarship',
 'other_info',
 'pros',
 'cons',
 'admission_preference',
 'admission_data_count',
 'admission_data',
 'application_notes',
 'other_notes']

前2行预览:
     program_name university region tier
0   Cambridge ACS  Cambridge     英国   T0
1  Cambridge MLMI  Cambridge     英国   T0


In [5]:
# Cell 5: 处理数据为ChromaDB格式
documents = []
metadatas = []
ids = []

print("🔄 处理项目数据...")

for index, row in df.iterrows():
    # 创建文档文本
    doc_text = create_document_text(row)
    documents.append(doc_text)
    
    # 创建元数据
    metadata = {
        'program_name': str(row['program_name']),
        'university': str(row['university']),
        'region': str(row['region']),
        'tier': str(row['tier']),
        'duration': str(row['duration']),
        'language': str(row['language']),
        'degree_type': str(row['degree_type']),
        'internship_required': True if str(row['internship_required']).strip() == '是' else False,
        'thesis_required': True if str(row['thesis_required']).strip() == '是' else False,
        'admission_data_count': int(row['admission_data_count']) if str(row['admission_data_count']).isdigit() else 0
    }
    metadatas.append(metadata)
    
    # 创建唯一ID
    ids.append(f"program_{index}")

print(f"✅ 成功处理 {len(documents)} 个项目")
print(f"📄 文档数量: {len(documents)}")
print(f"🏷️ 元数据数量: {len(metadatas)}")
print(f"🆔 ID数量: {len(ids)}")

🔄 处理项目数据...
✅ 成功处理 76 个项目
📄 文档数量: 76
🏷️ 元数据数量: 76
🆔 ID数量: 76


In [7]:
# Cell 6: 创建ChromaDB集合
# 删除现有集合（如果存在）
try:
    chroma_client.delete_collection(name=COLLECTION_NAME)
    print(f"🗑️ 删除现有集合: {COLLECTION_NAME}")
except:
    print(f"ℹ️ 集合 {COLLECTION_NAME} 不存在，将创建新集合")

# 创建新集合
collection = chroma_client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=default_ef
)
print(f"✅ 创建集合: {COLLECTION_NAME}")

🗑️ 删除现有集合: cs_programs_default
✅ 创建集合: cs_programs_default


In [8]:
# Cell 7: 批量添加数据到集合
batch_size = 10
total_batches = (len(documents) + batch_size - 1) // batch_size

print(f"📦 分 {total_batches} 批次添加数据，每批 {batch_size} 个项目")

for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    batch_metadata = metadatas[i:i+batch_size]
    
    batch_num = (i // batch_size) + 1
    
    try:
        collection.add(
            documents=batch_docs,
            metadatas=batch_metadata,
            ids=batch_ids
        )
        print(f"✅ 批次 {batch_num}/{total_batches} 添加成功")
        
    except Exception as e:
        print(f"❌ 批次 {batch_num} 添加失败: {str(e)}")
        break

print(f"🎉 集合创建完成，包含 {collection.count()} 个项目")

📦 分 8 批次添加数据，每批 10 个项目
✅ 批次 1/8 添加成功
✅ 批次 2/8 添加成功
✅ 批次 3/8 添加成功
✅ 批次 4/8 添加成功
✅ 批次 5/8 添加成功
✅ 批次 6/8 添加成功
✅ 批次 7/8 添加成功
✅ 批次 8/8 添加成功
🎉 集合创建完成，包含 76 个项目


In [17]:
# Cell 8: 查询辅助函数
def query_programs(query_text: str, n_results: int = 5, where_filter: Dict = None) -> Dict[str, Any]:
    """查询项目"""
    try:
        results = collection.query(
            query_texts=[query_text],
            n_results=n_results,
            where=where_filter
        )
        return results
    except Exception as e:
        print(f"❌ 查询失败: {e}")
        return {}

def print_query_results(results: Dict[str, Any], query_description: str = ""):
    """打印查询结果"""
    if not results or not results.get('documents') or not results['documents'][0]:
        print(f"\n🤷 {query_description} - 未找到符合条件的结果")
        return
    
    print(f"\n🔍 {query_description}")
    print("=" * 50)
    
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    )):
        print(f"\n📋 结果 {i+1} (向量距离: {distance:.3f}):")
        print(f"  项目: {metadata.get('program_name', 'N/A')}")
        print(f"  大学: {metadata.get('university', 'N/A')}")
        print(f"  地区: {metadata.get('region', 'N/A')}")
        print(f"  等级: {metadata.get('tier', 'N/A')}")
        print(f"  学制: {metadata.get('duration', 'N/A')}")
        print(f"  语言: {metadata.get('language', 'N/A')}")
        if metadata.get('admission_data_count', 0) > 0:
            print(f"  录取案例数: {metadata['admission_data_count']}")

print("✅ 查询辅助函数定义完成!")

✅ 查询辅助函数定义完成!


In [10]:
# Cell 9: 集合统计信息
all_data = collection.get()
total_count = len(all_data['metadatas'])

print(f"📊 集合统计信息")
print("=" * 30)
print(f"总项目数: {total_count}")

# 按地区统计
regions = [meta['region'] for meta in all_data['metadatas']]
region_counts = {}
for region in regions:
    region_counts[region] = region_counts.get(region, 0) + 1

print(f"\n📍 按地区分布:")
for region, count in sorted(region_counts.items()):
    print(f"  {region}: {count} 个项目")

# 按等级统计
tiers = [meta['tier'] for meta in all_data['metadatas']]
tier_counts = {}
for tier in tiers:
    tier_counts[tier] = tier_counts.get(tier, 0) + 1

print(f"\n🏆 按等级分布:")
for tier, count in sorted(tier_counts.items()):
    print(f"  {tier}: {count} 个项目")

# 论文要求统计
thesis_required = sum(1 for meta in all_data['metadatas'] if meta['thesis_required'])
print(f"\n📝 需要论文的项目: {thesis_required}/{total_count}")

📊 集合统计信息
总项目数: 76

📍 按地区分布:
  新加坡: 7 个项目
  日韩/其他: 9 个项目
  未分类: 5 个项目
  欧洲: 39 个项目
  英国: 12 个项目
  香港: 4 个项目

🏆 按等级分布:
  T0: 8 个项目
  T0.5: 1 个项目
  T1: 14 个项目
  T1.5: 24 个项目
  T2: 12 个项目
  TX: 17 个项目

📝 需要论文的项目: 46/76


In [18]:
# Cell 10: 查询示例1 - 短学制项目
results1 = query_programs(
    query_text="学制短",
    n_results=5,
    where_filter={"duration": {"$in": ["9个月", "10个月", "11个月", "12个月", "1年", "一年"]}}
)

print_query_results(results1, "短学制项目 (1年以内)")


🔍 短学制项目 (1年以内)

📋 结果 1 (向量距离: 1.304):
  项目: Oxford ACS
  大学: Oxford
  地区: 英国
  等级: T0
  学制: 一年
  语言: EN
  录取案例数: 1

📋 结果 2 (向量距离: 1.321):
  项目: NUS DSML
  大学: NUS
  地区: 新加坡
  等级: T2
  学制: 1年
  语言: EN
  录取案例数: 4

📋 结果 3 (向量距离: 1.342):
  项目: Cambridge ACS
  大学: Cambridge
  地区: 英国
  等级: T0
  学制: 9个月
  语言: EN
  录取案例数: 1

📋 结果 4 (向量距离: 1.403):
  项目: UCL CGVI
  大学: UCL
  地区: 英国
  等级: T1.5
  学制: 1年
  语言: EN
  录取案例数: 1

📋 结果 5 (向量距离: 1.444):
  项目: IC AC
  大学: IC
  地区: 英国
  等级: T1
  学制: 1年
  语言: EN
  录取案例数: 3


In [19]:
# Cell 11: 查询示例2 - 顶级项目
results2 = query_programs(
    query_text="顶级大学 计算机科学 人工智能",
    n_results=5,
    where_filter={"tier": {"$in": ["T0", "T1"]}}
)

print_query_results(results2, "顶级项目 (T0/T1)")


🔍 顶级项目 (T0/T1)

📋 结果 1 (向量距离: 1.059):
  项目: Aalto HCI
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 
  语言: 

📋 结果 2 (向量距离: 1.070):
  项目: Aalto CS
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 2年
  语言: EN

📋 结果 3 (向量距离: 1.102):
  项目: Polimi CSE
  大学: Polimi
  地区: 欧洲
  等级: T1
  学制: 2年
  语言: EN
  录取案例数: 1

📋 结果 4 (向量距离: 1.111):
  项目: Edinburgh AI
  大学: Edinburgh
  地区: 英国
  等级: T1
  学制: 
  语言: 

📋 结果 5 (向量距离: 1.131):
  项目: UvA AI
  大学: UvA
  地区: 欧洲
  等级: T1
  学制: 2 年
  语言: EN
  录取案例数: 1


In [25]:
# Cell 12: 查询示例3 - 陆本友好项目
results3 = query_programs(
    query_text="陆本友好",
    n_results=10
)

print_query_results(results3, "陆本友好项目")


🔍 陆本友好项目

📋 结果 1 (向量距离: 1.218):
  项目: UCL CS Msc
  大学: UCL
  地区: 英国
  等级: T1.5
  学制: 
  语言: 

📋 结果 2 (向量距离: 1.248):
  项目: UCL ML MSc
  大学: UCL
  地区: 英国
  等级: T1
  学制: 
  语言: 

📋 结果 3 (向量距离: 1.289):
  项目: UCL CSML MSc
  大学: UCL
  地区: 英国
  等级: T0
  学制: 
  语言: 

📋 结果 4 (向量距离: 1.289):
  项目: Edinburgh AI
  大学: Edinburgh
  地区: 英国
  等级: T1
  学制: 
  语言: 

📋 结果 5 (向量距离: 1.327):
  项目: Aalto MLDSAI
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 
  语言: 

📋 结果 6 (向量距离: 1.341):
  项目: UZH AI
  大学: UZH
  地区: 欧洲
  等级: TX
  学制: 
  语言: 

📋 结果 7 (向量距离: 1.342):
  项目: UCL DSML MSc
  大学: UCL
  地区: 英国
  等级: T1
  学制: 
  语言: 

📋 结果 8 (向量距离: 1.345):
  项目: Aalto HCI
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 
  语言: 

📋 结果 9 (向量距离: 1.350):
  项目: UZH DS
  大学: UZH
  地区: 欧洲
  等级: TX
  学制: 
  语言: 

📋 结果 10 (向量距离: 1.354):
  项目: Uppsala CS
  大学: Uppsala
  地区: 欧洲
  等级: T2
  学制: 2年
  语言: EN


In [21]:
# Cell 13: 查询示例4 - 不需要论文的项目
results4 = query_programs(
    query_text="不需要论文 coursework 授课型",
    n_results=3,
    where_filter={"thesis_required": False}
)

print_query_results(results4, "不需要论文的项目")


🔍 不需要论文的项目

📋 结果 1 (向量距离: 1.057):
  项目: UCL CSML MSc
  大学: UCL
  地区: 英国
  等级: T0
  学制: 
  语言: 

📋 结果 2 (向量距离: 1.088):
  项目: FAU AI
  大学: FAU
  地区: 欧洲
  等级: T2
  学制: 
  语言: 

📋 结果 3 (向量距离: 1.094):
  项目: UCL CS Msc
  大学: UCL
  地区: 英国
  等级: T1.5
  学制: 
  语言: 


In [22]:
# Cell 14: 查询示例5 - 奖学金项目
results5 = query_programs(
    query_text="奖学金 资助 经济支持 容易申请",
    n_results=5
)

print_query_results(results5, "有奖学金机会的项目")


🔍 有奖学金机会的项目

📋 结果 1 (向量距离: 0.834):
  项目: FAU AI
  大学: FAU
  地区: 欧洲
  等级: T2
  学制: 
  语言: 

📋 结果 2 (向量距离: 0.841):
  项目: KAUST CS Msc
  大学: KAUST
  地区: 日韩/其他
  等级: TX
  学制: 
  语言: EN
  录取案例数: 1

📋 结果 3 (向量距离: 0.843):
  项目: EM欧盟奖学金
  大学: EM欧盟奖学金
  地区: 未分类
  等级: T1.5
  学制: 2年
  语言: EN
  录取案例数: 1

📋 结果 4 (向量距离: 0.846):
  项目: CTH CAS
  大学: CTH
  地区: 欧洲
  等级: T2
  学制: 
  语言: 
  录取案例数: 1

📋 结果 5 (向量距离: 0.873):
  项目: Aalto HCI
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 
  语言: 


In [23]:
# Cell 15: 高级查询示例
print("🔍 执行高级查询示例")
print("=" * 40)

# 查询1: 英国T0级别项目
uk_t0_results = query_programs(
    query_text="计算机科学 人工智能 机器学习 顶级项目",
    n_results=3,
    where_filter={"$and": [{"region": "英国"}, {"tier": "T0"}]}
)
print_query_results(uk_t0_results, "英国T0级别项目")

# 查询2: 非英语国家的英语授课项目
non_english_countries = query_programs(
    query_text="英语授课 国际项目",
    n_results=3,
    where_filter={"$and": [
        {"language": "EN"}, 
        {"region": {"$nin": ["英国", "澳洲", "加拿大"]}}
    ]}
)
print_query_results(non_english_countries, "非英语国家的英语授课项目")

🔍 执行高级查询示例

🔍 英国T0级别项目

📋 结果 1 (向量距离: 0.900):
  项目: UCL CSML MSc
  大学: UCL
  地区: 英国
  等级: T0
  学制: 
  语言: 

📋 结果 2 (向量距离: 1.090):
  项目: Cambridge ACS
  大学: Cambridge
  地区: 英国
  等级: T0
  学制: 9个月
  语言: EN
  录取案例数: 1

📋 结果 3 (向量距离: 1.191):
  项目: Oxford ACS
  大学: Oxford
  地区: 英国
  等级: T0
  学制: 一年
  语言: EN
  录取案例数: 1

🔍 非英语国家的英语授课项目

📋 结果 1 (向量距离: 1.064):
  项目: DTU AI
  大学: DTU
  地区: 欧洲
  等级: T2
  学制: 2年
  语言: EN

📋 结果 2 (向量距离: 1.110):
  项目: Uppsala CS
  大学: Uppsala
  地区: 欧洲
  等级: T2
  学制: 2年
  语言: EN

📋 结果 3 (向量距离: 1.148):
  项目: University of Stuttgart CS
  大学: University
  地区: 欧洲
  等级: TX
  学制: 2年
  语言: EN
  录取案例数: 1


In [26]:
# Cell 16: 自定义查询区域
# 在这里可以输入自己的查询

# 示例：自定义查询
custom_query_text = "机器学习 深度学习 AI"  # 修改这里的查询文本
custom_filter = {"tier": {"$in": ["T0", "T1", "T1.5"]}}  # 修改这里的筛选条件
custom_n_results = 5  # 修改返回结果数量

custom_results = query_programs(
    query_text=custom_query_text,
    n_results=custom_n_results,
    where_filter=custom_filter
)

print_query_results(custom_results, f"自定义查询: '{custom_query_text}'")


🔍 自定义查询: '机器学习 深度学习 AI'

📋 结果 1 (向量距离: 1.123):
  项目: Edinburgh AI
  大学: Edinburgh
  地区: 英国
  等级: T1
  学制: 
  语言: 

📋 结果 2 (向量距离: 1.144):
  项目: Aalto HCI
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 
  语言: 

📋 结果 3 (向量距离: 1.150):
  项目: Aalto MLDSAI
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 
  语言: 

📋 结果 4 (向量距离: 1.208):
  项目: KU CS
  大学: KU
  地区: 欧洲
  等级: T1.5
  学制: 2年
  语言: EN
  录取案例数: 2

📋 结果 5 (向量距离: 1.213):
  项目: Aalto CS
  大学: Aalto
  地区: 欧洲
  等级: T1
  学制: 2年
  语言: EN


## 💡 使用提示

### 查询语法
- **基本查询**: 直接输入中文或英文关键词
- **筛选条件**: 使用 `where_filter` 参数进行精确筛选

### 常用筛选条件
```python
# 按地区筛选
{"region": "英国"}

# 按等级筛选
{"tier": {"$in": ["T0", "T1"]}}

# 按学制筛选
{"duration": {"$in": ["1年", "9个月"]}}

# 组合条件
{"$and": [{"region": "英国"}, {"tier": "T0"}]}

# 排除条件
{"region": {"$nin": ["英国", "澳洲"]}}
```

### 优势
- ✅ **无需API密钥**: 使用ChromaDB内置嵌入模型
- ✅ **离线使用**: 不依赖外部API服务
- ✅ **快速部署**: 直接运行即可使用
- ✅ **中文支持**: 支持中文查询和筛选

### 注意事项
- ⚠️ 默认嵌入模型的语义理解能力可能不如专业模型
- ⚠️ 建议使用具体的关键词进行查询
- ⚠️ 可以结合筛选条件提高查询精度